In [90]:
import re
import pandas as pd

# Baca file teks hasil eksport
file_path = "data_group.txt"  # Ubah dengan nama file Anda
with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

# Regex untuk memisahkan tanggal, waktu, pengirim, dan pesan
pattern = r"(\d{1,2}/\d{1,2}/\d{2,4}) (\d{1,2}\.\d{2}) - ([^:]+): (.+)"

# Parsing data
data = []
for line in lines:
    match = re.match(pattern, line)
    if match:
        date, time, sender, message = match.groups()
        data.append({"Date": date, "Time": time, "Sender": sender, "Message": message})

# Buat DataFrame
df = pd.DataFrame(data)

# Simpan ke file CSV
csv_file = "data_group.csv"
df.to_csv(csv_file, index=False)
print(f"Data telah disimpan ke file: {csv_file}")


Data telah disimpan ke file: data_group.csv


In [64]:
import tarfile
import pandas as pd
import re
from io import StringIO

# Fungsi untuk membaca file dari arsip .tar
def load_file_from_tar(tar_path, file_name):
    with tarfile.open(tar_path, 'r') as tar:
        # Mencari file di dalam arsip
        file = tar.extractfile(file_name)
        if file:
            return file.read().decode('utf-8')  # Mengembalikan data sebagai string
        else:
            raise FileNotFoundError(f"{file_name} tidak ditemukan dalam arsip.")

# Tentukan path arsip .tar dan nama file di dalam arsip
tar_file_path = "data_group.tar"
file_name_in_tar = "data_group.csv"  # Nama file di dalam arsip

# Membaca data dari arsip .tar
try:
    file_content = load_file_from_tar(tar_file_path, file_name_in_tar)
    
    # Tampilkan beberapa baris pertama dari file untuk debugging
    print("Beberapa baris pertama dari file yang diekstrak:")
    print(file_content[:500])  # Menampilkan 500 karakter pertama untuk pengecekan
    
    # Membaca data sebagai CSV
    file_data = StringIO(file_content)
    df = pd.read_csv(file_data)

    # Debug: Cek apakah kolom 'Message' ada
    if 'Message' not in df.columns:
        print("Kolom 'Message' tidak ditemukan!")
        print("Kolom yang tersedia:", df.columns)
    else:
        # Fungsi untuk membersihkan teks
        def clean_text(text):
            if isinstance(text, str):
                # Hanya mempertahankan angka, huruf, dan tanda baca-tulis umum
                return re.sub(r'[^a-zA-Z0-9.,:;!?()\-"\' ]', '', text)
            return text
        
        # Bersihkan kolom 'Message'
        df['Message'] = df['Message'].apply(clean_text)
        
        # Simpan hasil ke file CSV sementara
        output_csv_path = "data_group_cleaned.csv"
        df.to_csv(output_csv_path, index=False, encoding="utf-8")
        print(f"Data telah dibersihkan dan disimpan sementara ke file: {output_csv_path}")

        # Perbarui arsip .tar dengan file CSV hasil
        with tarfile.open(tar_file_path, "a") as tar:  # Gunakan mode "a" untuk menambahkan ke arsip yang ada
            tar.add(output_csv_path, arcname="data_group_cleaned.csv")
            print(f"File hasil telah ditambahkan ke arsip: {tar_file_path}")
    
except FileNotFoundError as e:
    print(f"Terjadi kesalahan: {e}")
except Exception as e:
    print(f"Terjadi kesalahan: {e}")

import os

# Hapus file sementara setelah digunakan
if os.path.exists(output_csv_path):
    os.remove(output_csv_path)
    print(f"File sementara {output_csv_path} telah dihapus.")
else:
    print(f"File sementara {output_csv_path} tidak ditemukan untuk dihapus.")



Beberapa baris pertama dari file yang diekstrak:
Date,Time,Sender,Message
27/08/22,07.12,+62 838-6608-5648,Pada ngumpul dimana nih?
27/08/22,07.23,Tofa IT,Pesan ini dihapus
27/08/22,07.23,+62 812-7846-0128,Di anu
27/08/22,08.25,+62 813-9016-2761,info sandi wifi edotarium kk?
29/08/22,16.29,+62 812-7846-0128,<Media tidak disertakan>
29/08/22,21.05,+62 812-7846-0128,ada ingpo pembagian kelompok LPPIK donk
29/08/22,21.18,Aldino Ti,Gek ndang satset
29/08/22,21.19,+62 838-6608-5648,<Media tidak disertakan>
29/08/22,21.39,+62 812-7846-0128,
Data telah dibersihkan dan disimpan sementara ke file: data_group_cleaned.csv
File hasil telah ditambahkan ke arsip: data_group.tar
File sementara data_group_cleaned.csv telah dihapus.


In [92]:
#scale.py
import tarfile
import pandas as pd
from itertools import islice
from sklearn.feature_extraction.text import CountVectorizer

def load_yelp_reviews(num_docs):
    # Membuka arsip .tar
    with tarfile.open('data_group.tar', 'r') as tar:
        # Mengakses file data_group_cleaned.csv di dalam arsip
        datafile = tar.extractfile('data_group_cleaned.csv')
        if datafile is None:
            raise FileNotFoundError("data_group_cleaned.csv tidak ditemukan dalam arsip data_group.tar")
        
        # Membaca file CSV ke dalam DataFrame
        df = pd.read_csv(datafile)
        
        # Memastikan kolom 'Message' ada
        if 'Message' not in df.columns:
            raise KeyError("Kolom 'Message' tidak ditemukan dalam file data_group_cleaned.csv")
        
        # Hilangkan baris dengan NaN di kolom 'Message'
        df['Message'] = df['Message'].fillna("").astype(str)
        
        # Mengambil sejumlah dokumen sesuai num_docs
        return list(islice(df['Message'], num_docs))

def make_matrix(docs, binary=False):
    # Modify min_df and max_df values
    vec = CountVectorizer(min_df=5, max_df=0.9, binary=binary)  # Adjusted parameters
    mtx = vec.fit_transform(docs)
    
    # Extract column names
    cols = [None] * len(vec.vocabulary_)
    for word, index in vec.vocabulary_.items():
        cols[index] = word
    
    return mtx, cols

# Contoh penggunaan
num_docs = 100
docs = load_yelp_reviews(num_docs)
mtx, cols = make_matrix(docs)

print("Matrix shape:", mtx.shape)
print("Feature names:", cols[:10])  # Menampilkan 10 fitur pertama


Matrix shape: (100, 9)
Feature names: ['ada', 'di', 'disertakan', 'info', 'infor', 'ini', 'media', 'tidak', 'turu']


In [66]:
import scipy.sparse

tar_file_path = "data_group.tar"

scipy.sparse.save_npz('matrix.npz', mtx)
pd.DataFrame({'feature': cols}).to_csv('features.csv', index=False)
print(f"Data Matrix disimpan ke file: {tar_file_path}")



Data Matrix disimpan ke file: data_group.tar


In [91]:
from itertools import islice
import numpy as np

def top_words(num_clusters, clusters, mtx, columns):
    top = []
    for i in range(num_clusters):  # Loop over each cluster
        rows_in_cluster = np.where(clusters == i)[0]  # Get rows belonging to the current cluster
        word_freqs = mtx[rows_in_cluster].sum(axis=0).A[0]  # Sum word frequencies for the cluster
        ordered_freqs = np.argsort(word_freqs)  # Sort frequencies
        top_words = [
            (columns[idx], int(word_freqs[idx]))  # Get top words and their frequencies
            for idx in islice(reversed(ordered_freqs), 20)
        ]
        top.append(top_words)
    return top


In [93]:
import tarfile
from itertools import islice
from sklearn.feature_extraction.text import CountVectorizer

# Fungsi untuk membaca file dari arsip .tar
def load_custom_data(file_path, file_name, num_docs):
    with tarfile.open(file_path, 'r') as tar:
        # Membuka file tertentu di dalam arsip
        datafile = tar.extractfile(file_name)
        if datafile:
            # Mengembalikan baris pertama sejumlah num_docs
            return list(islice(datafile, num_docs))
        else:
            raise FileNotFoundError(f"{file_name} tidak ditemukan dalam arsip.")

# Fungsi untuk membuat matriks fitur dari teks
def make_matrix(docs, binary=False):
    vec = CountVectorizer(min_df=1, max_df=0.9, binary=binary)
    mtx = vec.fit_transform(docs)
    cols = [None] * len(vec.vocabulary_)
    for word, idx in vec.vocabulary_.items():
        cols[idx] = word
    return mtx, cols

file_path = 'data_group.tar'
file_name = 'data_group.csv'
num_docs = 100 

# Membaca data
try:
    docs = load_custom_data(file_path, file_name, num_docs)
    # Mengubah byte ke string (jika diperlukan)
    docs = [doc.decode('utf-8').strip() for doc in docs]

    # Membuat matriks fitur
    matrix, columns = make_matrix(docs)
    print("Matriks fitur dibuat dengan shape:", matrix.shape)
    print("Contoh kolom fitur:", columns[:10])
except Exception as e:
    print("Terjadi kesalahan:", e)

Matriks fitur dibuat dengan shape: (100, 329)
Contoh kolom fitur: ['00', '0028', '005', '01', '0128', '0268', '05', '06', '07', '0868']


In [86]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import tarfile
import os

# Fungsi untuk membaca file dari arsip .tar
def extract_csv_from_tar(tar_path, file_name):
    with tarfile.open(tar_path, 'r') as tar:
        # Memeriksa apakah file yang dimaksud ada di dalam arsip
        if file_name in tar.getnames():
            print(f"Mengekstrak {file_name} dari {tar_path}...")
            extracted_file = tar.extractfile(file_name)
            if extracted_file:
                return pd.read_csv(extracted_file)
        else:
            raise FileNotFoundError(f"{file_name} tidak ditemukan di dalam arsip {tar_path}.")

# Path ke file tar dan nama file CSV di dalamnya
tar_path = 'data_group.tar'
file_name = 'data_group_cleaned.csv'

# Membaca file CSV dari arsip .tar
try:
    data = extract_csv_from_tar(tar_path, file_name)
except Exception as e:
    print(f"Terjadi kesalahan: {e}")
    exit()

# Pastikan kolom 'Message' ada
if 'Message' not in data.columns:
    raise KeyError("Kolom 'Message' tidak ditemukan di dataset!")

# Step 1: Preprocess the text data (cleaning the "Message" column)
data['Message'] = data['Message'].fillna("").str.lower()  # Fill NaN and convert to lowercase

# Step 2: Convert the "Message" column into numerical representation using TF-IDF
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X = vectorizer.fit_transform(data['Message'])

# Step 3: Perform KMeans clustering with 3, 4, and 5 clusters
clusters = {}
for n_clusters in [3, 4, 5]:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters[n_clusters] = kmeans.fit_predict(X)

# Add cluster labels to the dataset for each KMeans run
for n_clusters, labels in clusters.items():
    data[f'Cluster_{n_clusters}'] = labels

# Save the updated dataset to a new CSV file
output_file = 'data_group_clustered.csv'
data.to_csv(output_file, index=False)
print(f"Data dengan kluster telah disimpan ke file: {output_file}")

# Step 4: Add the resulting CSV file to the tar archive
def add_file_to_tar(tar_path, file_to_add):
    with tarfile.open(tar_path, 'a') as tar:  # Open in append mode
        tar.add(file_to_add, arcname=os.path.basename(file_to_add))
        print(f"File {file_to_add} telah ditambahkan ke arsip {tar_path}.")

try:
    add_file_to_tar(tar_path, output_file)
except Exception as e:
    print(f"Terjadi kesalahan saat menambahkan file ke arsip: {e}")


import os

# Hapus file sementara setelah digunakan
if os.path.exists(output_file):
    os.remove(output_file)
    print(f"File sementara {output_file} telah dihapus.")
else:
    print(f"File sementara {output_file} tidak ditemukan untuk dihapus.")

Mengekstrak data_group_cleaned.csv dari data_group.tar...
Data dengan kluster telah disimpan ke file: data_group_clustered.csv
File data_group_clustered.csv telah ditambahkan ke arsip data_group.tar.
File sementara data_group_clustered.csv telah dihapus.


In [87]:
import tarfile
import pandas as pd
from collections import Counter

# Fungsi untuk membaca file dari arsip .tar
def extract_csv_from_tar(tar_path, file_name):
    with tarfile.open(tar_path, 'r') as tar:
        # Memeriksa apakah file yang dimaksud ada di dalam arsip
        if file_name in tar.getnames():
            print(f"Mengekstrak {file_name} dari {tar_path}...")
            extracted_file = tar.extractfile(file_name)
            if extracted_file:
                return pd.read_csv(extracted_file)
        else:
            raise FileNotFoundError(f"{file_name} tidak ditemukan di dalam arsip {tar_path}.")

# Path ke file tar dan nama file CSV di dalamnya
tar_path = 'data_group.tar'
file_name = 'data_group_clustered.csv'

# Membaca file CSV dari arsip .tar
try:
    data = extract_csv_from_tar(tar_path, file_name)
except Exception as e:
    print(f"Terjadi kesalahan: {e}")
    exit()

# Pastikan kolom 'Message' dan 'Cluster_3' ada
if 'Message' not in data.columns or 'Cluster_3' not in data.columns:
    raise KeyError("Kolom 'Message' atau 'Cluster_3' tidak ditemukan di dataset!")

# Mendapatkan 3 kata teratas untuk setiap kluster
for cluster_id in sorted(data['Cluster_3'].unique()):  # Iterasi berdasarkan kluster
    # Mengambil pesan-pesan dari kluster tertentu
    cluster_messages = data[data['Cluster_3'] == cluster_id]['Message']
    
    # Mengonversi pesan menjadi string dan mengganti NaN dengan string kosong
    cluster_messages = cluster_messages.fillna("").astype(str)
    
    # Menggabungkan semua pesan dalam kluster menjadi satu string dan memisahkannya menjadi kata-kata
    all_words = " ".join(cluster_messages).split()
    
    # Mendapatkan 3 kata teratas menggunakan Counter
    most_common_words = Counter(all_words).most_common(3)
    
    # Menampilkan hasil
    print(f"Kluster {cluster_id}: {most_common_words}")


Mengekstrak data_group_clustered.csv dari data_group.tar...
Kluster 0: [('ini', 161), ('pesan', 95), ('dihapus', 93)]
Kluster 1: [('media', 592), ('tidak', 592), ('disertakan', 592)]
Kluster 2: [('terimakasih', 8), ('kak', 2), ('sudah', 2)]


In [89]:
import tarfile
import pandas as pd
from collections import Counter
import re

# Fungsi untuk membaca file hasil klastering dari tar
def read_clustered_data_from_tar(tar_file_path, csv_file_name):
    with tarfile.open(tar_file_path, "r") as tar:
        extracted_file = tar.extractfile(csv_file_name)
        if extracted_file:
            df = pd.read_csv(extracted_file)
            return df
        else:
            raise FileNotFoundError(f"{csv_file_name} not found in {tar_file_path}")

# Fungsi untuk mendapatkan 3 kata teratas dari setiap kluster
def analyze_clusters(data, cluster_column, message_column):
    analysis = []
    for cluster_id in data[cluster_column].unique():
        cluster_messages = data[data[cluster_column] == cluster_id][message_column].fillna("").astype(str)
        all_words = " ".join(cluster_messages).split()
        clean_words = [re.sub(r'[^a-zA-Z0-9]', '', word).lower() for word in all_words if len(word) > 1]
        most_common_words = Counter(clean_words).most_common(3)
        analysis.append({
            "Cluster": cluster_id,
            "Top Words": ", ".join([f"{word} ({count})" for word, count in most_common_words])
        })
    return pd.DataFrame(analysis)

# File TAR dan nama file hasil klastering di dalamnya
tar_file_path = "data_group.tar"
clustered_file_name = "data_group_clustered.csv"

# Analisis data hasil klastering
try:
    df = read_clustered_data_from_tar(tar_file_path, clustered_file_name)
    cluster_analysis = analyze_clusters(df, cluster_column="Cluster_3", message_column="Message")

    # Simpan hasil analisis ke CSV
    analysis_csv_path = "cluster_analysis.csv"
    cluster_analysis.to_csv(analysis_csv_path, index=False, encoding="utf-8")
    print(f"Hasil analisis telah disimpan ke {analysis_csv_path}")
    print(cluster_analysis)
except Exception as e:
    print(f"Terjadi kesalahan: {e}")



Hasil analisis telah disimpan ke cluster_analysis.csv
   Cluster                                   Top Words
0        0         ini (162), pesan (95), dihapus (93)
1        1  media (592), tidak (592), disertakan (592)
2        2         terimakasih (8), kak (2), sudah (2)
